In [1]:
import numpy as np
import cv2

mask = cv2.imread('00157-lbl.png',1)
img = cv2.imread('00157.png',1)
mask = cv2.cvtColor(mask, cv2.COLOR_BGR2GRAY)
ret,thresh = cv2.threshold(mask,127,255,0)
im2 ,contours, hierarchy = cv2.findContours(thresh,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
cimg = cv2.drawContours(img,contours,-1,(0,255,0),1)


# cv2.imshow('teg', cimg)
# cv2.waitKey(0)
# cv2.destroyAllWindows()

In [2]:
def flattener(image, pts, w, h):
    """Flattens an image of a card into a top-down 200x300 perspective.
    Returns the flattened, re-sized, grayed image.
    See www.pyimagesearch.com/2014/08/25/4-point-opencv-getperspective-transform-example/"""
    temp_rect = np.zeros((4,2), dtype = "float32")
    
    s = np.sum(pts, axis = 2)

    tl = pts[np.argmin(s)]
    br = pts[np.argmax(s)]

    diff = np.diff(pts, axis = -1)
    tr = pts[np.argmin(diff)]
    bl = pts[np.argmax(diff)]

    # Need to create an array listing points in order of
    # [top left, top right, bottom right, bottom left]
    # before doing the perspective transform

    if w <= 0.8*h: # If card is vertically oriented
        temp_rect[0] = tl
        temp_rect[1] = tr
        temp_rect[2] = br
        temp_rect[3] = bl

    if w >= 1.2*h: # If card is horizontally oriented
        temp_rect[0] = bl
        temp_rect[1] = tl
        temp_rect[2] = tr
        temp_rect[3] = br

    # If the card is 'diamond' oriented, a different algorithm
    # has to be used to identify which point is top left, top right
    # bottom left, and bottom right.
    
    if w > 0.8*h and w < 1.2*h: #If card is diamond oriented
        # If furthest left point is higher than furthest right point,
        # card is tilted to the left.
        if pts[1][0][1] <= pts[3][0][1]:
            # If card is titled to the left, approxPolyDP returns points
            # in this order: top right, top left, bottom left, bottom right
            temp_rect[0] = pts[1][0] # Top left
            temp_rect[1] = pts[0][0] # Top right
            temp_rect[2] = pts[3][0] # Bottom right
            temp_rect[3] = pts[2][0] # Bottom left

        # If furthest left point is lower than furthest right point,
        # card is tilted to the right
        if pts[1][0][1] > pts[3][0][1]:
            # If card is titled to the right, approxPolyDP returns points
            # in this order: top left, bottom left, bottom right, top right
            temp_rect[0] = pts[0][0] # Top left
            temp_rect[1] = pts[3][0] # Top right
            temp_rect[2] = pts[2][0] # Bottom right
            temp_rect[3] = pts[1][0] # Bottom left
            
        
    maxWidth = 200
    maxHeight = 300

    # Create destination array, calculate perspective transform matrix,
    # and warp card image
    dst = np.array([[0,0],[maxWidth-1,0],[maxWidth-1,maxHeight-1],[0, maxHeight-1]], np.float32)
    M = cv2.getPerspectiveTransform(temp_rect,dst)
    warp = cv2.warpPerspective(image, M, (maxWidth, maxHeight))
    warp = cv2.cvtColor(warp,cv2.COLOR_BGR2GRAY)

        

    return warp

In [3]:
peri = cv2.arcLength(contours[0], True)
approx = cv2.approxPolyDP(contours[0], 0.09*peri, True)
pts = np.float32(approx)
x,y,w,h = cv2.boundingRect(contours[0])
average = np.sum(pts, axis=0)/len(pts)
cent_x = int(average[0][0])
cent_y =  int(average[0][1])
warp = flattener(img, pts, w, h)

In [4]:
ret,thresh = cv2.threshold(warp,127,255,cv2.THRESH_BINARY)
im2 ,contours, hierarchy = cv2.findContours(thresh,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)

In [10]:
len(contours[0])

36

In [20]:
area = np.array([cv2.contourArea(contours[i]) for i in range(len(contours))]) 
len(area[area > 1000])

9

In [5]:
# print(len(contours))
# cimg = cv2.drawContours(img,contours,-1,(0,255,0),1)
# cv2.imshow('teg', cimg)
# cv2.waitKey(0)
# cv2.destroyAllWindows()

In [6]:
# cv2.imshow('teg', warp)
# cv2.waitKey(0)
# cv2.destroyAllWindows()